In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from abc import ABCMeta, abstractmethod
from time import time

In [81]:
class Function(object) :
    def __init__(self):
        pass

    def forward(self , x ) :
        pass

    def backward(self , y) :
        pass

In [82]:
class sigmoid(Function) :
    #@staticmethod
    def forward (self , x) :
        return 1. / (1. + np.exp(-x))
    #@staticmethod
    def backward (self , x) :
        return x * (1. - x)

In [83]:
class Fx(Function) :
    #@staticmethod
    def forward (self , x) :
        return x 
    #@staticmethod
    def backward (self , x) :
        return 1.

In [466]:
class Node(object) :
    def __init__(self , nin , name , func = Fx() ) :
        self.nin = nin
        self.win = np.random.random(len(nin)) * 1e-3
        
        for i in range(len(nin)) :
            self.nin[i].nout.append(self)
            self.nin[i].wout.append(self.win[i])
            
        self.nout = []
        self.wout = []
        
        self.value = None
        self.loss = None
        self.func = func
        self.name = name
    def forward(self) :
        if self.value :
            return self.value
        s = 0
        for i in range(len(self.nin)) :
            s += self.nin[i].forward() * self.win[i]
        self.value = self.func.forward(s)
        return self.value
    def backward(self) :
        if self.loss :
            return self.loss
        l = 0
        for i in range(len(self.wout)) :
            l += self.nout[i].backward() * self.wout[i]
            self.nout[i].win[ self.nout[i].nin.index(self) ] -= 0.01 * self.nout[i].backward()
        self.loss = self.func.backward(l) * l 
        
        return self.loss
    def update(self) :
        self.value = self.value if isinstance(self , Holder) else None
        self.loss = None
        for i in range(len(self.nin)) :
            self.nin[i].update()
            ## update wout
            self.nin[i].wout[self.nin[i].nout.index(self)] = self.win[i]
            
    @property
    def status(self) :
        print("name : {}".format(self.name))
        for i in range(len(self.win)) :
            print("{} -|{:.5}|-> ".format(self.nin[i].name , self.win[i]))
        for i in range(len(self.wout)) :
            print("--|{:.5}|->{} ".format(self.wout[i] , self.nout[i].name))

In [467]:
class Holder(Node):
    def __init__(self , name , func = Fx()) :
        self.nin = []
        self.win = []
            
        self.nout = []
        self.wout = []
        
        self.name = name
        self.value = None
        self.loss = None
        self.func = func
    def forward(self) :
        assert self.value
        return self.value

In [468]:
def setHolder( n , x ) :
    assert len(n) == len(x)
    for i in range(len(n)) :
        n[i].value = x[i]

In [609]:
#input layer
x = [Holder('x-{}'.format(i) , sigmoid()) for i in range(3)]

#hidden layer
n1 = [Node(x , 'n1-{}'.format(i) , sigmoid() ) for i in range(5) ]

#hidden layer2
n2 = [Node(n1 , 'n2-{}'.format(i) , sigmoid() ) for i in range(3) ]

#output layer
y = Node(n2 , 'y-0' , sigmoid())


In [610]:
n1[1].status

name : n1-1
x-0 -|0.00043197|-> 
x-1 -|0.00063623|-> 
x-2 -|0.00082262|-> 
--|0.00095976|->n2-0 
--|0.00023417|->n2-1 
--|8.3655e-05|->n2-2 


In [611]:
setHolder(x , np.ones(3) ) 

In [614]:
for iter in range(1001) :
    y.forward()
    y.loss = y.func.backward(y.forward() - 0.1 )
    for i in x :
        i.backward() 
    if iter%100 == 0 :
        print(iter , y.value , y.loss)
    y.update()

0 0.11456928859408551 0.014357024423947753
100 0.112871215624623 0.012705547432967458
200 0.11137708157381775 0.011247643588680438
300 0.1100608596088771 0.009959638712807558
400 0.10890024324039779 0.00882102891065954
500 0.1078760111562617 0.007813979604528141
600 0.10697152607815211 0.006922923902293747
700 0.10617233359057655 0.006134235888623188
800 0.10546583672910485 0.005435961357955616
900 0.10484102886970124 0.004817593309183958
1000 0.10428827215317686 0.004269882875117144


In [615]:
x[1].status

name : x-1
--|0.00096293|->n1-0 
--|0.00062726|->n1-1 
--|0.00095905|->n1-2 
--|0.00043324|->n1-3 
--|8.8602e-05|->n1-4 


In [616]:
y.status

name : y-0
n2-0 -|-1.6104|-> 
n2-1 -|-1.6103|-> 
n2-2 -|-1.6108|-> 


In [522]:
x_ = [[1,1],[1,0],[0,1],[0,0]]
y_ = [1,1,1,0]

In [523]:
#input layer
x = [Holder('x-{}'.format(i) , sigmoid()) for i in range(2)]

#hidden layer
n = [Node(x , 'n-{}'.format(i) , sigmoid() ) for i in range(3) ]

#output layer
y = Node(n , 'y-0' , sigmoid())

In [592]:
y.status

name : y-0
n-0 -|0.23293|-> 
n-1 -|0.23268|-> 
n-2 -|0.23274|-> 


In [593]:
for iter in range(1) :
    setHolder(x , x_[iter%4] ) 
    y.forward()
    y.loss = y.func.backward(y.forward() - y_[iter%4] )
    for i in x :
        i.backward() 
    if iter%1000 == 0 :
        print(iter , y.value , y.loss)
    y.update()

0 0.5862254683195153 -0.5849838947478891


In [477]:
x_[iter%4]

[1, 0]

In [478]:
x[0].value